In [2]:
out_dir  <- file.path('/mnt/18T/chibao/gliomas/data/upstream/scRNA/official/integrated_v5_optimized/adult/subclusters/myeloid/GSVA')

In [7]:
library(tidyverse)
library(ComplexHeatmap)

# Load mean GSVA matrix (pathways x cell_types)
gsva_mean <- read.csv(file.path(out_dir, "tables/gsva_REACTOME_mean_by_celltype.csv"),
                      row.names = 1, check.names = FALSE)

# ---- OPTION 1: Manually list the pathways you want to show ----
# Put your curated pathways here (exact rownames from the CSV)
canonical_paths <- c(
    "REACTOME_ANTIGEN_PROCESSING_UB_ATP_INDEPENDENT_PROTEASOMAL_DEGRADATION",
    "REACTOME_ANTIGEN_PROCESSING_CROSS_PRESENTATION",
    "REACTOME_MHC_CLASS_II_ANTIGEN_PRESENTATION",
    "REACTOME_RESPIRATORY_ELECTRON_TRANSPORT",
    "REACTOME_INTERFERON_ALPHA_BETA_SIGNALING",
    "REACTOME_INTERFERON_GAMMA_SIGNALING",
    "REACTOME_EVASION_BY_RSV_OF_HOST_INTERFERON_RESPONSES",
    "REACTOME_INITIAL_TRIGGERING_OF_COMPLEMENT",
    "REACTOME_COMPLEMENT_CASCADE",
    "REACTOME_NF_KB_IS_ACTIVATED_AND_SIGNALS_SURVIVAL",
    "REACTOME_CHEMOKINE_RECEPTORS_BIND_CHEMOKINES",
    "REACTOME_RIP_MEDIATED_NFKB_ACTIVATION_VIA_ZBP1",
    "REACTOME_CELLULAR_RESPONSE_TO_HYPOXIA",
    "REACTOME_REGULATION_OF_APOPTOSIS",
    "REACTOME_RESPIRATORY_SYNCYTIAL_VIRUS_RSV_GENOME_REPLICATION_TRANSCRIPTION_AND_TRANSLATION",
    "REACTOME_REGULATION_OF_GENE_EXPRESSION_BY_HYPOXIA_INDUCIBLE_FACTOR",
    "REACTOME_FCGAMMA_RECEPTOR_FCGR_DEPENDENT_PHAGOCYTOSIS",
    "REACTOME_FCERI_MEDIATED_NF_KB_ACTIVATION",
    "REACTOME_ROLE_OF_PHOSPHOLIPIDS_IN_PHAGOCYTOSIS",
    "REACTOME_ROS_AND_RNS_PRODUCTION_IN_PHAGOCYTES",
    "REACTOME_ANTIGEN_ACTIVATES_B_CELL_RECEPTOR_BCR_LEADING_TO_GENERATION_OF_SECOND_MESSENGERS",
    "REACTOME_CROSS_PRESENTATION_OF_SOLUBLE_EXOGENOUS_ANTIGENS_ENDOSOMES"
)

# Keep only those that exist
canonical_paths <- intersect(canonical_paths, rownames(gsva_mean))
if (length(canonical_paths) == 0) stop("None of your canonical_paths matched rownames(gsva_mean).")

mat <- as.matrix(gsva_mean[canonical_paths, , drop = FALSE])

# Z-score per pathway across cell types (improves contrast)
mat_z <- t(scale(t(mat)))

# pdf(file.path(out_dir, "plots/heatmap_REACTOME_CANONICAL_manual.pdf"), width = 30, height = 14)
# Heatmap(mat_z, name = "Mean GSVA (z)", cluster_rows = TRUE, cluster_columns = TRUE)
# dev.off()

In [ ]:
library(tidyverse)
library(ComplexHeatmap)
library(circlize)
library(grid)

# Load mean GSVA matrix (pathways x cell_types)
gsva_mean <- read.csv(
  file.path(out_dir, "tables/gsva_REACTOME_mean_by_celltype.csv"),
  row.names = 1, check.names = FALSE
)

# Curated pathways
canonical_paths <- c(
  "REACTOME_ANTIGEN_PROCESSING_UB_ATP_INDEPENDENT_PROTEASOMAL_DEGRADATION",
  "REACTOME_ANTIGEN_PROCESSING_CROSS_PRESENTATION",
  "REACTOME_MHC_CLASS_II_ANTIGEN_PRESENTATION",
  "REACTOME_RESPIRATORY_ELECTRON_TRANSPORT",
  "REACTOME_INTERFERON_ALPHA_BETA_SIGNALING",
  "REACTOME_INTERFERON_GAMMA_SIGNALING",
  "REACTOME_EVASION_BY_RSV_OF_HOST_INTERFERON_RESPONSES",
  "REACTOME_INITIAL_TRIGGERING_OF_COMPLEMENT",
  "REACTOME_COMPLEMENT_CASCADE",
  "REACTOME_NF_KB_IS_ACTIVATED_AND_SIGNALS_SURVIVAL",
  "REACTOME_CHEMOKINE_RECEPTORS_BIND_CHEMOKINES",
  "REACTOME_RIP_MEDIATED_NFKB_ACTIVATION_VIA_ZBP1",
  "REACTOME_CELLULAR_RESPONSE_TO_HYPOXIA",
  "REACTOME_REGULATION_OF_APOPTOSIS",
  "REACTOME_RESPIRATORY_SYNCYTIAL_VIRUS_RSV_GENOME_REPLICATION_TRANSCRIPTION_AND_TRANSLATION",
  "REACTOME_REGULATION_OF_GENE_EXPRESSION_BY_HYPOXIA_INDUCIBLE_FACTOR",
  "REACTOME_FCGAMMA_RECEPTOR_FCGR_DEPENDENT_PHAGOCYTOSIS",
  "REACTOME_FCERI_MEDIATED_NF_KB_ACTIVATION",
  "REACTOME_ROLE_OF_PHOSPHOLIPIDS_IN_PHAGOCYTOSIS",
  "REACTOME_ROS_AND_RNS_PRODUCTION_IN_PHAGOCYTES",
  "REACTOME_ANTIGEN_ACTIVATES_B_CELL_RECEPTOR_BCR_LEADING_TO_GENERATION_OF_SECOND_MESSENGERS",
  "REACTOME_CROSS_PRESENTATION_OF_SOLUBLE_EXOGENOUS_ANTIGENS_ENDOSOMES"
)

canonical_paths <- intersect(canonical_paths, rownames(gsva_mean))
if (length(canonical_paths) == 0) stop("None of your canonical_paths matched rownames(gsva_mean).")

mat <- as.matrix(gsva_mean[canonical_paths, , drop = FALSE])

# Row z-score (relative enrichment across cell types)
mat_z <- t(scale(t(mat)))

# -------------------------
# 1) Optional: enforce a biologically meaningful column order
# -------------------------
# Set this to your preferred order (must match colnames(mat_z)).
# If you leave it NULL, we'll keep the current order.
preferred_col_order <- NULL
# preferred_col_order <- c("Monocyte","Neutrophil","MERT_TAM","C1Q_MHCII_TAM","IFN_TAM",
#                          "Inflammatory_TAM","Hypoxic_TAM","SPP1_TAM","HSP_TAM")

if (!is.null(preferred_col_order)) {
  preferred_col_order <- intersect(preferred_col_order, colnames(mat_z))
  mat_z <- mat_z[, preferred_col_order, drop = FALSE]
}

# -------------------------
# 2) Split rows into functional blocks (this is what makes it “advanced” fast)
# -------------------------
row_group <- case_when(
  str_detect(rownames(mat_z), "ANTIGEN|MHC|CROSS_PRESENTATION|PROTEASOMAL") ~ "Antigen / MHC / Cross-presentation",
  str_detect(rownames(mat_z), "INTERFERON|EVASION|RSV") ~ "Interferon / Antiviral",
  str_detect(rownames(mat_z), "COMPLEMENT") ~ "Complement",
  str_detect(rownames(mat_z), "NF_KB|NFKB|CHEMOKINE|RIP_MEDIATED") ~ "NFκB / Chemokine",
  str_detect(rownames(mat_z), "HYPOXIA|HIF") ~ "Hypoxia / HIF",
  str_detect(rownames(mat_z), "APOPTOSIS") ~ "Apoptosis",
  str_detect(rownames(mat_z), "PHAGOCYTOSIS|FCGAMMA|FCERI|PHOSPHOLIPIDS|ROS|RNS") ~ "Phagocytosis / ROS",
  str_detect(rownames(mat_z), "RESPIRATORY_ELECTRON_TRANSPORT") ~ "Mitochondrial respiration",
  TRUE ~ "Other"
)
row_split <- factor(row_group, levels = unique(row_group))

# Order rows within each block by “most specific” (max abs z across columns)
row_order <- order(row_split, -apply(abs(mat_z), 1, max))
mat_z <- mat_z[row_order, , drop = FALSE]
row_split <- row_split[row_order]

# -------------------------
# 3) Better color mapping (quantile-based so outliers don’t crush contrast)
# -------------------------
q <- quantile(mat_z, probs = c(0.02, 0.50, 0.98), na.rm = TRUE)
col_fun <- circlize::colorRamp2(c(q[1], q[2], q[3]), c("#2C7BB6", "white", "#D7191C"))

# -------------------------
# 4) Typography + sizing controls
# -------------------------
row_font <- 18
col_font <- 16
title_font <- 19
legend_font <- 11

# Size the heatmap body in physical units (helps “control cell size”)
# Wider columns -> increase width; taller rows -> increase height
heatmap_width  <- unit(12, "cm")
heatmap_height <- unit(18, "cm")

ht <- Heatmap(
  mat_z,
  name = "Mean GSVA (row z)",
  col = col_fun,

  # Structure
  row_split = row_split,
  cluster_rows = FALSE,           # you already impose interpretable grouping/order
  cluster_columns = TRUE,         # keep clustering columns if you want patterns; set FALSE to freeze order
  show_row_dend = FALSE,

  # Labels
  row_names_side = "left",
  row_names_gp = gpar(fontsize = row_font),
  column_names_gp = gpar(fontsize = col_font),
  column_names_rot = 45,

  # Titles
  column_title = "Myeloid states",
  row_title = "",
  column_title_gp = gpar(fontsize = title_font, fontface = "bold"),
  #row_title_gp = gpar(fontsize = title_font, fontface = "bold"),

  # Legend
  heatmap_legend_param = list(
    title_gp = gpar(fontsize = legend_font, fontface = "bold"),
    labels_gp = gpar(fontsize = legend_font),
    legend_height = unit(4, "cm")
  ),

  # Aesthetics
  rect_gp = gpar(col = "grey85", lwd = 0.4),  # subtle cell borders
  border = TRUE,

  # Performance / export
  use_raster = TRUE,              # makes PDF smaller & faster
  raster_quality = 4,

  width = heatmap_width,
  height = heatmap_height
)

pdf(file.path(out_dir, "plots/heatmap_REACTOME_CANONICAL_manual_advanced.pdf"),
    width = 20, height = 10)  # PDF page size; heatmap size is controlled by width/height above
draw(ht, heatmap_legend_side = "right", annotation_legend_side = "right")
dev.off()


In [8]:
library(stringr)

# --- 1) Define a mapping: old Reactome name -> short label ---
path_label_map <- c(
  "REACTOME_ANTIGEN_PROCESSING_UB_ATP_INDEPENDENT_PROTEASOMAL_DEGRADATION" =
    "Ag processing (Ub/ATP-indep proteasome)",
  "REACTOME_ANTIGEN_PROCESSING_CROSS_PRESENTATION" =
    "Ag processing: cross-presentation",
  "REACTOME_MHC_CLASS_II_ANTIGEN_PRESENTATION" =
    "MHC-II antigen presentation",
  "REACTOME_RESPIRATORY_ELECTRON_TRANSPORT" =
    "Respiratory electron transport",

  "REACTOME_INTERFERON_ALPHA_BETA_SIGNALING" =
    "IFN (α/β) signaling",
  "REACTOME_INTERFERON_GAMMA_SIGNALING" =
    "IFN (γ) signaling",
  "REACTOME_EVASION_BY_RSV_OF_HOST_INTERFERON_RESPONSES" =
    "Antiviral IFN evasion",

  "REACTOME_INITIAL_TRIGGERING_OF_COMPLEMENT" =
    "Complement initiation",
  "REACTOME_COMPLEMENT_CASCADE" =
    "Complement cascade",

  "REACTOME_NF_KB_IS_ACTIVATED_AND_SIGNALS_SURVIVAL" =
    "NF-κB survival signaling",
  "REACTOME_RIP_MEDIATED_NFKB_ACTIVATION_VIA_ZBP1" =
    "NF-κB activation via ZBP1",
  "REACTOME_CHEMOKINE_RECEPTORS_BIND_CHEMOKINES" =
    "Chemokine receptor signaling",

  "REACTOME_CELLULAR_RESPONSE_TO_HYPOXIA" =
    "Hypoxia response",
  "REACTOME_REGULATION_OF_GENE_EXPRESSION_BY_HYPOXIA_INDUCIBLE_FACTOR" =
    "HIF-regulated transcription",
  "REACTOME_REGULATION_OF_APOPTOSIS" =
    "Apoptosis regulation",

  "REACTOME_RESPIRATORY_SYNCYTIAL_VIRUS_RSV_GENOME_REPLICATION_TRANSCRIPTION_AND_TRANSLATION" =
    "Viral replication",

  "REACTOME_FCGAMMA_RECEPTOR_FCGR_DEPENDENT_PHAGOCYTOSIS" =
    "FcγR-mediated phagocytosis",
  "REACTOME_FCERI_MEDIATED_NF_KB_ACTIVATION" =
    "FcεRI-mediated NF-κB activation",
  "REACTOME_ROLE_OF_PHOSPHOLIPIDS_IN_PHAGOCYTOSIS" =
    "Phospholipids in phagocytosis",
  "REACTOME_ROS_AND_RNS_PRODUCTION_IN_PHAGOCYTES" =
    "ROS/RNS in phagocytes",

  "REACTOME_ANTIGEN_ACTIVATES_B_CELL_RECEPTOR_BCR_LEADING_TO_GENERATION_OF_SECOND_MESSENGERS" =
    "BCR signaling",
  "REACTOME_CROSS_PRESENTATION_OF_SOLUBLE_EXOGENOUS_ANTIGENS_ENDOSOMES" =
    "Endosomal cross-presentation"
)

# --- 2) Build display labels in the same row order as mat_z ---
row_ids <- rownames(mat_z)

# Keep original if you didn't define a label for some row
row_labels <- ifelse(row_ids %in% names(path_label_map),
                     path_label_map[row_ids],
                     row_ids)

# Ensure uniqueness (ComplexHeatmap doesn’t like duplicated labels)
row_labels <- make.unique(row_labels)

# Quick “how much did we shorten?” example (numbers)
message("Median original label length: ", median(nchar(row_ids)))
message("Median short label length:    ", median(nchar(row_labels)))

# --- 3) Use the short labels ONLY for display by replacing rownames in a copy ---
mat_z_disp <- mat_z
rownames(mat_z_disp) <- row_labels

Median original label length: 45.5

Median short label length:    24.5



In [ ]:
library(tidyverse)
library(ComplexHeatmap)
library(circlize)
library(grid)

# Assuming gsva_mean, canonical_paths, and mat_z are loaded/created
# and out_dir is defined from the prior context.
# We will use the mat_z created in the previous step,
# but ensure the final display uses the shortened labels.

# --- 1) Define a mapping: old Reactome name -> short label (Already done) ---
# ... (path_label_map definition is complete)

# --- 2) Build display labels in the same row order as mat_z (Already done) ---
row_ids <- rownames(mat_z) # mat_z should be the canonical_paths matrix, z-scored
row_labels <- ifelse(row_ids %in% names(path_label_map),
                     path_label_map[row_ids],
                     row_ids)
row_labels <- make.unique(row_labels) # Ensure uniqueness

# --- 3) Create display matrix mat_z_disp (Already done) ---
mat_z_disp <- mat_z
rownames(mat_z_disp) <- row_labels

# -------------------------
# 1) Optional: enforce a biologically meaningful column order
# -------------------------
preferred_col_order <- NULL
if (!is.null(preferred_col_order)) {
  preferred_col_order <- intersect(preferred_col_order, colnames(mat_z))
  mat_z <- mat_z[, preferred_col_order, drop = FALSE]
  # IMPORTANT: Apply to the display matrix too
  mat_z_disp <- mat_z_disp[, preferred_col_order, drop = FALSE]
}

# -------------------------
# 2) Split rows into functional blocks (uses ORIGINAL row names in mat_z)
# -------------------------
row_group <- case_when(
  str_detect(rownames(mat_z), "ANTIGEN|MHC|CROSS_PRESENTATION|PROTEASOMAL") ~ "Antigen / MHC / Cross-presentation",
  str_detect(rownames(mat_z), "INTERFERON|EVASION|RSV") ~ "Interferon / Antiviral",
  str_detect(rownames(mat_z), "COMPLEMENT") ~ "Complement",
  str_detect(rownames(mat_z), "NF_KB|NFKB|CHEMOKINE|RIP_MEDIATED") ~ "NFκB / Chemokine",
  str_detect(rownames(mat_z), "HYPOXIA|HIF") ~ "Hypoxia / HIF",
  str_detect(rownames(mat_z), "APOPTOSIS") ~ "Apoptosis",
  str_detect(rownames(mat_z), "PHAGOCYTOSIS|FCGAMMA|FCERI|PHOSPHOLIPIDS|ROS|RNS") ~ "Phagocytosis / ROS",
  str_detect(rownames(mat_z), "RESPIRATORY_ELECTRON_TRANSPORT") ~ "Mitochondrial respiration",
  TRUE ~ "Other"
)
row_split <- factor(row_group, levels = unique(row_group))

# Order rows within each block by “most specific” (max abs z across columns)
row_order <- order(row_split, -apply(abs(mat_z), 1, max))

# === CRITICAL MODIFICATION: Apply row order to BOTH matrices and the split vector ===
mat_z <- mat_z[row_order, , drop = FALSE]
mat_z_disp <- mat_z_disp[row_order, , drop = FALSE] # Use the new matrix with short labels
row_split <- row_split[row_order]

# -------------------------
# 3) Better color mapping (quantile-based so outliers don’t crush contrast)
# -------------------------
q <- quantile(mat_z, probs = c(0.02, 0.50, 0.98), na.rm = TRUE)
col_fun <- circlize::colorRamp2(c(q[1], q[2], q[3]), c("#2C7BB6", "white", "#D7191C"))

# -------------------------
# 4) Typography + sizing controls
# -------------------------
row_font <- 20
col_font <- 20
title_font <- 19
legend_font <- 11

heatmap_width  <- unit(12, "cm")
heatmap_height <- unit(18, "cm")

ht <- Heatmap(
  # === CRITICAL MODIFICATION: Use the display matrix here ===
  mat_z_disp,
  name = "Mean GSVA",
  col = col_fun,

  # Structure
  row_split = row_split,
  cluster_rows = FALSE,
  cluster_columns = TRUE,
  show_row_dend = FALSE,
  gap = unit(4, "mm"),

  # Labels
  row_names_side = "left",
  row_names_gp = gpar(fontsize = row_font),
  column_names_gp = gpar(fontsize = col_font),
  column_names_rot = 90,

  # Titles
  column_title = "Myeloid states",
  row_title = NULL, # Hidden as requested
  column_title_gp = gpar(fontsize = title_font, fontface = "bold"),

  # Legend
  heatmap_legend_param = list(
    title_gp = gpar(fontsize = legend_font, fontface = "bold"),
    labels_gp = gpar(fontsize = legend_font),
    legend_height = unit(4, "cm")
  ),

  # Aesthetics
  rect_gp = gpar(col = "grey85", lwd = 0.4),
  border = TRUE,

  # Performance / export
  use_raster = TRUE,
  raster_quality = 4,

  width = heatmap_width,
  height = heatmap_height
)

png(file.path(out_dir, "plots/heatmap_REACTOME_CANONICAL_manual_advanced_shortlabels.png"),
    width = 6000, height = 4000, res = 300) # Use 2000x1000 pixels at 300 DPI for high quality
draw(ht, heatmap_legend_side = "right", annotation_legend_side = "right")
dev.off()

ERROR: Error: object 'row_split' not found
